#### Задача 1 (3 балла). 

Разовьем тему с бойцами. Напишите игру (можно взять свой старый код в качестве базы), где игроку будет предложено выбрать класс героя: волшебник или боец. Вы должны учесть возможность добавления новых игровых классов (используйте наследование). У волшебника и бойца немного разные атрибуты (можно атрибуты сделать одинаковые в классе-родителе, но разные коэффициенты в классах-детях, на которые они домножаются: например, здоровье волшебника будет 1.0 от стандартного значения, а здоровье бойца - 1.5, а с маной наоборот). Также у них будут разные методы "нанести удар" и, если хотите, приветствия. Также у нашего героя, кем бы он ни был, должен быть рюкзак, в котором можно рыться и хранить ограниченный набор вещей (в частности, там хранятся зелья: по умолчанию пусть в начале игры каждому персонажу дается по три зелья). Наконец, нужен класс для противника: можете придумать любого монстра (тоже с возможностью добавления новых монстров, очевидно), с которым герой будет сражаться. Во время сражения неплохо предоставлять игроку выбор вида "нанести удар - выпить зелье", а сам урон от удара можно немного рандомизировать с помощью одноименного модуля. Можно еще реализовать и метод sleep для мирного времени, но во время боя он, конечно, не понадобится. 

In [78]:
import random


class Hero:
    base_health = 100
    base_mana = 30
    base_attack = 20
    base_defense = 10

    def __init__(self, name, health_multiplier=1.0, mana_multiplier=1.0, 
                 attack_multiplier=1.0, defense_multiplier=1.0):
        self.name = name
        self.max_health = int(self.base_health * health_multiplier)
        self.health = self.max_health
        self.max_mana = int(self.base_mana * mana_multiplier)
        self.mana = self.max_mana
        self.attack_power = int(self.base_attack * attack_multiplier)
        self.defense = int(self.base_defense * defense_multiplier)
        self.backpack = Backpack()
    
    def attack(self, enemy):
        damage = int(max(0, self.attack_power - enemy.defense) * random.uniform(0.9, 1.1))
        enemy.health -= damage
        print(f'{self.name} наносит удар {enemy.name} на {damage} урона.')

    def greet(self):
        print(f'{self.name} готов к приключениям.')

    def sleep(self):
        print(f'{self.name} спит.')
        health_restore = int(self.max_health * 0.2)
        mana_restore = int(self.max_mana * 0.2)
        self.health = min(self.max_health, self.health + health_restore)
        self.mana = min(self.max_mana, self.mana + mana_restore)
        print(f'{self.name} восстановил {health_restore} здоровья и {mana_restore} маны.')
        self.status()

    def is_alive(self):
        return self.health > 0
    
    def status(self):
        print(f'{self.name} Здоровье: {self.health}/{self.max_health} | Мана: {self.mana}/{self.max_mana}')


class Fighter(Hero):
    def __init__(self, name):
        super().__init__(name, health_multiplier=1.5, mana_multiplier=0.5, attack_multiplier=1.2, defense_multiplier=1.1)


class Wizard(Hero):
    def __init__(self, name):
        super().__init__(name, health_multiplier=1.0, mana_multiplier=2.0, attack_multiplier=0.8, defense_multiplier=0.9)
    
        for _ in range(3):
            self.backpack.add_item(Potion('Зелье маны', 'mana', 30))

    def attack(self, enemy):
        if self.mana >= 10:
            self.mana -= 10
            damage = int((self.attack_power + 10) * random.uniform(0.9, 1.1)) 
            enemy.health -= damage
            print(f'Волшебник {self.name} наносит магический удар {enemy.name} на {damage} урона. Мана: {self.mana}/{self.max_mana}')
        else:
            print('Недостаточно маны для магической атаки.')
            super().attack(enemy) # обычная атака


class Monster:
    def __init__(self, name, health, mana, attack_power, defense, loot=None):
        self.name = name
        self.health = health
        self.mana = mana
        self.attack_power = attack_power
        self.defense = defense
        self.loot = loot if loot else []

    def is_alive(self):
        return self.health > 0

    def attack(self, target):
        damage = int(max(0, self.attack_power - target.defense) * random.uniform(0.9, 1.1))
        target.health -= damage
        print(f'{self.name} наносит удар {target.name} на {damage} урона.')
        if not target.is_alive():
            print(f'{target.name} погибает от удара {self.name}!')

    def greet(self):
        print(f'{self.name} готов атаковать!')
    
    def drop_loot(self, hero):
        if self.loot:
            print(f"Вы нашли: {', '.join(item.name for item in self.loot)}")
            for item in self.loot:
                hero.backpack.add_item(item)


class SkeletonWarrior(Monster):
    def __init__(self):
        super().__init__(name='Скелет-воин', health=50, mana=0, attack_power=20, defense=5, loot=[Potion('Зелье здоровья', 'health', 20)])


class DarkKnight(Monster):
    def __init__(self):
        super().__init__(name='Темный рыцарь', health=100, mana=0, attack_power=25, defense=10, loot=[Potion('Зелье здоровья', 'health', 30)])


class Item:
    def __init__(self, name):
        self.name = name

    def use(self, character):
        pass


class Potion(Item):
    def __init__(self, name, potion_type, restore_amount):
        super().__init__(name)
        self.potion_type = potion_type # 'health' или 'mana'
        self.restore_amount = restore_amount

    def use(self, character):
        if self.potion_type == 'health':
            character.health = min(character.max_health, character.health + self.restore_amount)
            print(f'{character.name} восстановил {self.restore_amount} здоровья.')
        elif self.potion_type == 'mana':
            character.mana = min(character.max_mana, character.mana + self.restore_amount)
            print(f'{character.name} восстановил {self.restore_amount} маны.')
        else:
            print('Неизвестный тип зелья.')


class Backpack:
    def __init__(self, slots=10):
        self.slots = slots
        self.items = []

    def add_item(self, item):
        if len(self.items) < self.slots:
            self.items.append(item)
            print(f'Добавлено: {item.name}')
        else:
            print('В рюкзаке нет места')

    def remove_item(self, index):
        if 0 <= index < len(self.items):
            removed_item = self.items.pop(index)
            print(f'Удалено: {removed_item.name}')
            return removed_item
        else:
            print('Неверный номер предмета.')
            return None

    def use_item(self, index, character):
        if 0 <= index < len(self.items):
            item = self.items[index]
            item.use(character)
            self.remove_item(index)
        else:
            print('Неверный номер предмета.')

    def __iter__(self):
        return BackpackIterator(self)

    def list_items(self):
        if not self.items:
            print('Рюкзак пуст.')
            return
        print('Рюкзак:')
        for idx, item in enumerate(self, 1):
            print(f'{idx}. {item.name}')


class BackpackIterator:
    def __init__(self, backpack):
        self.backpack = backpack
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.index < len(self.backpack.items):
            item = self.backpack.items[self.index]
            self.index += 1
            return item
        else:
            raise StopIteration


class Game:
    def __init__(self):
        self.hero = None
        self.monster_classes = [SkeletonWarrior, DarkKnight]

    def run(self):
        print('Древняя крепость пала под натиском темной магии. Ее мертвые стражи ожили и бродят по руинам, охраняя забытые тайны.')
        self.choose_class()
        self.hero.greet()

        while True:
            print('\nВыберите действие:')
            print('1. Исследовать руины крепости')
            print('2. Посмотреть инвентарь')
            print('3. Спать')
            print('4. Выйти из игры')
            choice = input('Ваш выбор: ')

            if choice == '1':
                self.explore()
            elif choice == '2':
                self.open_inventory()
            elif choice == '3':
                self.hero.sleep()
            elif choice == '4':
                print('\nИгра завершена')
                break
            else:
                print('Неверный выбор. Попробуйте снова.')

            if not self.hero.is_alive():
                print('Вы погибли. Игра окончена.')
                break

    def choose_class(self):
        while True:
            print('Выберите класс героя:')
            print('1. Боец')
            print('2. Волшебник')
            choice = input('Введите номер класса: ')
            if choice == '1':
                name = input('Введите имя бойца: ')
                self.hero = Fighter(name)
                break
            elif choice == '2':
                name = input('Введите имя волшебника: ')
                self.hero = Wizard(name)
                break
            else:
                print('Неверный выбор. Попробуйте снова.')
    
    def explore(self):
        print('\nВы исследуете крепость...')
        if random.random() < 0.7: # 70% шанс встретить монстра
            monster = self.generate_monster()
            print(f'Вы столкнулись с {monster.name}')
            self.battle(monster)
        else:
            print('Вы ничего не нашли')
    
    def generate_monster(self):
        monster_class = random.choice(self.monster_classes)
        return monster_class()

    def battle(self, monster):
        print(f'\n{self.hero.name} VS {monster.name}')
        monster.greet()
        while self.hero.is_alive() and monster.is_alive():
            self.hero.status()
            print('\nВаш ход')
            print('1. Нанести удар')
            print('2. Использовать зелье')
            action = input('Ваш выбор: ')

            if action == '1':
                self.hero.attack(monster)
            elif action == '2':
                self.open_inventory()
            else:
                print('Неверный выбор. Попробуйте снова.')
                continue

            if monster.is_alive():
                print('\nХод монстра')
                monster.attack(self.hero)
            else:
                print(f'{self.hero.name} победил {monster.name}!')
                monster.drop_loot(self.hero)

    def open_inventory(self):
        while True:
            print('\nИнвентарь')
            self.hero.backpack.list_items()
            if not self.hero.backpack.items:
                break
            print('Выберите действие:')
            print('1. Использовать зелье')
            print('2. Выйти из инвентаря')
            choice = input('Ваш выбор: ')
            if choice == '1':
                potion_number = int(input('Введите номер зелья: '))
                if 1 <= potion_number <= len(self.hero.backpack.items):
                    self.hero.backpack.use_item(potion_number - 1, self.hero)
                else:
                    print('Неверный номер зелья.')
            elif choice == '2':
                break
            else:
                print('Неверный выбор. Попробуйте снова.')


game = Game()
game.run()


Древняя крепость пала под натиском темной магии. Ее мертвые стражи ожили и бродят по руинам, охраняя забытые тайны.
Выберите класс героя:
1. Боец
2. Волшебник
Добавлено: Зелье маны
Добавлено: Зелье маны
Добавлено: Зелье маны
Wizard готов к приключениям.

Выберите действие:
1. Исследовать руины крепости
2. Посмотреть инвентарь
3. Спать
4. Выйти из игры

Вы исследуете крепость...
Вы столкнулись с Скелет-воин

Wizard VS Скелет-воин
Скелет-воин готов атаковать!
Wizard Здоровье: 100/100 | Мана: 60/60

Ваш ход
1. Нанести удар
2. Использовать зелье
Волшебник Wizard наносит магический удар Скелет-воин на 23 урона. Мана: 50/60

Ход монстра
Скелет-воин наносит удар Wizard на 10 урона.
Wizard Здоровье: 90/100 | Мана: 50/60

Ваш ход
1. Нанести удар
2. Использовать зелье
Волшебник Wizard наносит магический удар Скелет-воин на 25 урона. Мана: 40/60

Ход монстра
Скелет-воин наносит удар Wizard на 11 урона.
Wizard Здоровье: 79/100 | Мана: 40/60

Ваш ход
1. Нанести удар
2. Использовать зелье
Волшебник 

#### Задача 2 (3 балла). 

Вспомним задачу токенизации. Напишите собственный простенький токенизатор (с самим процессом можно не сильно заморачиваться), который будет создавать генератор с объектами класса Token, у которых будет атрибут text и атрибут category (латинское слово, кириллическое слово или пунктуация). Токенизатор должен быть реализован в классе, у которого должна быть (генераторная) функция tokenize(). Вам понадобится отдельный класс для токенов и re.finditer(). 

In [67]:
import re

class Token:
    def __init__(self, text, category):
        self.text = text
        self.category = category

class Tokenizer:
    def __init__(self, text):
        self.text = text

    def tokenize(self):
        pattern = r'[a-zA-Z]+|[а-яА-ЯёЁ]+|[^\w\s]+'
        for match in re.finditer(pattern, self.text):
            matched_text = match.group()
            if re.fullmatch(r'[a-zA-Z]+', matched_text):
                category = 'латинское слово'
            elif re.fullmatch(r'[а-яА-ЯёЁ]+', matched_text):
                category = 'кириллическое слово'
            else:
                category = 'пунктуация'
            yield Token(matched_text, category)

text = 'Регулярные выражения (англ. regular expressions) — формальный язык, используемый в компьютерных программах, работающих с текстом, для поиска и осуществления манипуляций с подстроками в тексте, основанный на использовании метасимволов (символов-джокеров, англ. wildcard characters).'
tokenizer = Tokenizer(text)

for token in tokenizer.tokenize():
    print(f'text: {token.text:<25} category: {token.category}')


text: Регулярные                category: кириллическое слово
text: выражения                 category: кириллическое слово
text: (                         category: пунктуация
text: англ                      category: кириллическое слово
text: .                         category: пунктуация
text: regular                   category: латинское слово
text: expressions               category: латинское слово
text: )                         category: пунктуация
text: —                         category: пунктуация
text: формальный                category: кириллическое слово
text: язык                      category: кириллическое слово
text: ,                         category: пунктуация
text: используемый              category: кириллическое слово
text: в                         category: кириллическое слово
text: компьютерных              category: кириллическое слово
text: программах                category: кириллическое слово
text: ,                         category: пунктуация
text: ра